<a href="https://colab.research.google.com/github/sdikici/Demand_Forecasting_Prophet_DeepAR/blob/main/DeepAR_hyperparameter_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle datasets download -d sercandikici/merged-dataset-electricty-weather-for-modelling
! unzip merged-dataset-electricty-weather-for-modelling.zip

Dataset URL: https://www.kaggle.com/datasets/sercandikici/merged-dataset-electricty-weather-for-modelling
License(s): unknown
  0% 0.00/104k [00:00<?, ?B/s]
100% 104k/104k [00:00<00:00, 89.3MB/s]
Archive:  merged-dataset-electricty-weather-for-modelling.zip
  inflating: merged_data.csv         


In [8]:
!pip install gluonts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 23.7 MB/s eta 0:00:00


In [9]:
pip install mxnet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 31.4 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.3
    Uninstalling graphviz-0.20.3:
      Successfully uninstalled graphviz-0.20.3


In [10]:
pip install mxnet --upgrade

In [11]:
import numpy as np
np.bool = np.bool_

In [12]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import mxnet as mx
from mxnet import gluon
import json
from gluonts.dataset.common import ListDataset
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.mx import DeepAREstimator
from gluonts.mx.trainer import Trainer
import itertools

/usr/local/lib/python3.10/dist-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [13]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true))
    return mape

def root_mean_squared_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mse = np.mean((y_true - y_pred) ** 2)
    rmse = np.sqrt(mse)
    return rmse

def r_squared(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mean_y_true = np.mean(y_true)
    ss_total = np.sum((y_true - mean_y_true) ** 2)
    ss_residual = np.sum((y_true - y_pred) ** 2)
    r2 = 1 - (ss_residual / ss_total)
    return r2

In [14]:
mx.random.seed(7)
np.random.seed(7)

In [15]:
df_merged = pd.read_csv("merged_data.csv")
df_merged['settlement_date'] = pd.to_datetime(df_merged['settlement_date'])

In [16]:
df_model_3 = df_merged.set_index("settlement_date")

split_from = 90*12 #train test split is from 90days

train_data = df_model_3[:-split_from]
test_data = df_model_3[-split_from:]

In [19]:
freq = "2H"
days_to_predict=30
prediction_length = days_to_predict*12 #predicting 7 days
context_length = prediction_length *2

In [22]:
# Define the parameter grid for DeepAR
param_grid = {
    'epochs': [5, 10],
    'num_batches_per_epoch': [50, 100],
    'num_cells': [40, 64],
    'num_layers': [2,4],
    'dropout_rate':[0.01, 0.05],
    'learning_rate':[0.001, 0.0001]
}

# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
mapes = []  # Store the MAPEs for each parameter combination here

# Use cross validation to evaluate all parameters
for params in all_params:
    # Define DeepAR estimator with given parameters
    estimator = DeepAREstimator(
        freq=freq,
        context_length=context_length,
        prediction_length=prediction_length,
        dropout_rate = params['dropout_rate'],
        use_feat_dynamic_real =True, # weather temp
        num_layers=params['num_layers'],
        num_cells=params['num_cells'],
        trainer=Trainer(
            epochs=params['epochs'],
            num_batches_per_epoch=params['num_batches_per_epoch'],
            learning_rate=params['learning_rate']
        )
    )

    train_ds = ListDataset(
    [{"start":train_data.index[0],
      "target":train_data.tsd,
      "feat_dynamic_real": [train_data.temp],
      "feat_dynamic_cat": [train_data.is_holiday],
      }],
    freq=freq
)

    test_ds = ListDataset(
        [{"start":test_data.index[0],
          "target":test_data.tsd,
          "feat_dynamic_real": [test_data.temp],
          "feat_dynamic_cat": [test_data.is_holiday],
          }],
        freq=freq
    )

    # Train the model
    m = estimator.train(training_data=train_ds)
    # Generate forecasts
    forecast_it, ts_it = make_evaluation_predictions(
        dataset=test_ds,
        predictor=m,
        num_samples=100
    )
    forecasts = list(forecast_it)
    tss = list(ts_it)

    # Get MAPE for the forecasts
    evaluator = Evaluator()
    agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts))
    mape = agg_metrics["MAPE"]
    mapes.append(mape)

# Combine parameters and corresponding MAPEs into a DataFrame
tuning_results = pd.DataFrame(all_params)
tuning_results['mape'] = mapes
print(tuning_results)


100%|██████████| 50/50 [01:29<00:00,  1.80s/it, epoch=10/10, avg_epoch_loss=8.4] 
Running evaluation: 1it [00:00,  5.10it/s]
100%|██████████| 50/50 [01:31<00:00,  1.83s/it, epoch=10/10, avg_epoch_loss=8.44]
Running evaluation: 1it [00:00,  4.61it/s]
100%|██████████| 50/50 [01:45<00:00,  2.11s/it, epoch=10/10, avg_epoch_loss=8.44]
Running evaluation: 1it [00:00,  3.88it/s]
100%|██████████| 50/50 [01:41<00:00,  2.03s/it, epoch=10/10, avg_epoch_loss=8.51]
Running evaluation: 1it [00:00,  3.49it/s]
100%|██████████| 100/100 [02:58<00:00,  1.78s/it, epoch=10/10, avg_epoch_loss=8.23]
Running evaluation: 1it [00:00,  3.20it/s]
100%|██████████| 100/100 [03:03<00:00,  1.83s/it, epoch=10/10, avg_epoch_loss=8.24]
Running evaluation: 1it [00:00,  2.95it/s]
100%|██████████| 100/100 [03:30<00:00,  2.10s/it, epoch=10/10, avg_epoch_loss=8.1]
Running evaluation: 1it [00:00,  2.84it/s]
100%|██████████| 100/100 [03:29<00:00,  2.09s/it, epoch=10/10, avg_epoch_loss=8.21]
Running evaluation: 1it [00:00,  2.7

    epochs  num_batches_per_epoch  num_cells  num_layers  dropout_rate  \
0       10                     50         40           2          0.01   
1       10                     50         40           2          0.05   
2       10                     50         64           2          0.01   
3       10                     50         64           2          0.05   
4       10                    100         40           2          0.01   
5       10                    100         40           2          0.05   
6       10                    100         64           2          0.01   
7       10                    100         64           2          0.05   
8       15                     50         40           2          0.01   
9       15                     50         40           2          0.05   
10      15                     50         64           2          0.01   
11      15                     50         64           2          0.05   
12      15                    100     

In [23]:
tuning_results

,epochs,num_batches_per_epoch,num_cells,num_layers,dropout_rate,mape
0,10,50,40,2,0.01,0.119122
1,10,50,40,2,0.05,0.111769
2,10,50,64,2,0.01,0.092166
3,10,50,64,2,0.05,0.071987
4,10,100,40,2,0.01,0.194206
5,10,100,40,2,0.05,0.309596
6,10,100,64,2,0.01,0.113400
7,10,100,64,2,0.05,0.269508
8,15,50,40,2,0.01,0.261658
9,15,50,40,2,0.05,0.090507
